In [1]:
!pip install snowflake-connector-python bcrypt
!pip install -U langchain-openai

**Import and Load**

In [3]:
import os
import pandas as pd
import bcrypt
import gradio as gr
import snowflake.connector

import config
import configy

from openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

**Enter key here or use config file sent by development**

In [5]:
client = OpenAI(api_key=configy.OPENAI_AI_KEY)

**Load exported csv**

In [7]:
books = pd.read_csv("cleaned_books.csv")

In [8]:
books.sample(5)

,isbn13,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
4728,9780802137784,Last Words,William S. Burroughs,Biography/Memoir,http://books.google.com/books/content?id=gRxjT...,Documenting the final months of this literary ...,2001.0,3.98,273.0,479.0
796,9780140443875,Terre,Émile Zola,Other,http://books.google.com/books/content?id=JaTP2...,No Marketing Blurb,1980.0,4.08,499.0,1053.0
5010,9780816736911,You Can Never Go Home Again,Dyan Sheldon,Self-Help,http://books.google.com/books/content?id=b9fDU...,When Angel and her mother move into a cottage ...,1995.0,4.26,166.0,21.0
6128,9781598163322,"BLOOD SUCKER Volume 1: Legend of ""Zipangu""",Saki Okuse;Aki Shimizu,Other,http://books.google.com/books/content?id=wIj-I...,"With the help of his right hand man Kuraha, Mi...",2006.0,3.20,200.0,49.0
5886,9781579126261,The Body in the Library,Agatha Christie,Other,http://books.google.com/books/content?id=hcgee...,Miss Marple investigates the death of a stylis...,2006.0,3.85,191.0,43669.0


**Create Embeddings for every description using Langchain in order to give the dimensions for vector search to work**

In [10]:
loader = TextLoader("code_description.txt", encoding="utf-8")
raw_documents = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    separator="\n"
)
documents = text_splitter.split_documents(raw_documents)

# Create embeddings using the API key
embeddings = OpenAIEmbeddings(openai_api_key=configy.OPENAI_AI_KEY)

# Create Chroma vector store
db_books = Chroma.from_documents(
    documents,
    embedding=embeddings
)


Created a chunk of size 1170, which is longer than the specified 500
Created a chunk of size 1216, which is longer than the specified 500
Created a chunk of size 962, which is longer than the specified 500
Created a chunk of size 845, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1191, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 754, which is longer than the specified 500
Created a chunk of size 730, which is longer than the specified 500
Created a chunk of size 723, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 683, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 523, which is longer than the specified 500
Created a chunk of size 789, which is longer

**Snowflake connection for password auth**

In [12]:
def get_snowflake_connection():
    return snowflake.connector.connect(
        user=config.SNOWFLAKE_USER,
        password=config.SNOWFLAKE_PASSWORD,
        account=config.SNOWFLAKE_ACCOUNT,
        warehouse=config.SNOWFLAKE_WAREHOUSE,
        database=config.SNOWFLAKE_DATABASE,
        schema=config.SNOWFLAKE_SCHEMA
    )

**Functions for creating an account and sending those values to snowflake database**

In [14]:
# Account creation
def create_account(username, password):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    try:
        cs.execute(
            "INSERT INTO USER_ACCOUNTS (username, password_hash) VALUES (%s, %s)",
            (username, hashed_pw)
        )
        conn.commit()
        return "Account created successfully."
    except snowflake.connector.errors.ProgrammingError as e:
        if "unique constraint" in str(e).lower():
            return "Username already exists."
        else:
            return f"Error: {e}"
    finally:
        cs.close()
        conn.close()

# Login
def login(username, password):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "SELECT password_hash FROM USER_ACCOUNTS WHERE username = %s",
            (username,)
        )
        row = cs.fetchone()
        if row:
            stored_hash = row[0].encode()
            if bcrypt.checkpw(password.encode(), stored_hash):
                return True, "Login successful."
            else:
                return False, "Invalid password."
        else:
            return False, "User not found."
    except Exception as e:
        return False, f"Error: {e}"
    finally:
        cs.close()
        conn.close()

**Functions for Vector seach that finds book titles based on how closely related the prompts are to the description of the book giving a top 5**

In [16]:
# Retrieve recommendations
def retrieve_recommendations(query: str, top_k: int = 5) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=top_k)
    books_list = []

    for rec in recs:
        try:
            isbn_str = rec.page_content.split()[0].strip("'").strip('"')
            isbn_num = int(isbn_str)
            books_list.append(isbn_num)
        except Exception as e:
            print(f"Skipping invalid record: {e}")

    if not books_list:
        return pd.DataFrame()

    return books[books["isbn13"].isin(books_list)].head(top_k)

# Recommend books
def recommend_books(description, category):
    query_parts = [description]
    if category != "All":
        query_parts.append(f"Category: {category}")
    query = ". ".join(query_parts)

    df = retrieve_recommendations(query, top_k=5)
    if df.empty:
        return "No recommendations found.", gr.update(choices=[])

    result_text = ""
    dropdown_choices = []
    for _, row in df.iterrows():
        label = f"{row['title']} by {row['authors']} (ISBN: {row['isbn13']})"
        dropdown_choices.append(label)
        result_text += (
            f"**{row['title']}** by {row['authors']}\n\n"
            f"**Genre:** {row['categories']}\n"
            f"**Average Rating:** {row['average_rating']}\n\n"
            f"{row['description'][:200]}...\n\n---\n\n"
        )
    return result_text, gr.update(choices=dropdown_choices, value=None)
    
def format_reading_list(username):
    df = get_reading_list(username)
    if df.empty:
        return "Your reading list is empty."

    text = "### Your Reading List:\n\n"
    for _, row in df.iterrows():
        text += (
            f"- **{row['title']}** by {row['authors']} (ISBN: {row['isbn13']})\n"
        )
    return text


**Login and Sign Up interface for gradio**

In [18]:
def login_fn(username, password):
    success, msg = login(username, password)
    if success:
        return (
            gr.update(visible=False),  # Hide login
            gr.update(visible=False),  # Hide signup
            gr.update(visible=True),   # Show app
            msg
        )
    else:
        return (
            gr.update(visible=True),
            gr.update(visible=True),
            gr.update(visible=False),
            msg
        )

def signup_fn(new_username, new_password):
    try:
        msg = create_account(new_username, new_password)
        return msg
    except Exception as e:
        return f"Error: {e}"

def add_to_reading_list(username, isbn13):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "INSERT INTO BOOKAPPDB.PUBLIC.USER_READING_LIST (username, isbn13) VALUES (%s, %s)",
            (username, isbn13)
        )
        conn.commit()
        return f"Book {isbn13} added to your reading list."
    except Exception as e:
        return f"Error adding book: {e}"
    finally:
        cs.close()
        conn.close()


def get_reading_list(username):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "SELECT isbn13 FROM BOOKAPPDB.PUBLIC.USER_READING_LIST WHERE username = %s ORDER BY added_at DESC",
            (username,)
        )
        rows = cs.fetchall()
        if not rows:
            return pd.DataFrame()
        isbn_list = [r[0] for r in rows]
        return books[books["isbn13"].isin(isbn_list)]
    except Exception as e:
        print(f"Error retrieving reading list: {e}")
        return pd.DataFrame()
    finally:
        cs.close()
        conn.close()


**Gradio Interface**

In [20]:
# Install dependencies (run in your notebook or terminal)
# !pip install snowflake-connector-python bcrypt
# !pip install -U langchain-openai

import os
import pandas as pd
import bcrypt
import gradio as gr
import snowflake.connector

import config
import configy

from openai import OpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Initialize OpenAI client
client = OpenAI(api_key=configy.OPENAI_AI_KEY)

# Load books data
books = pd.read_csv("cleaned_books.csv")

# Load and embed descriptions
loader = TextLoader("code_description.txt", encoding="utf-8")
raw_documents = loader.load()

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    separator="\n"
)
documents = text_splitter.split_documents(raw_documents)

embeddings = OpenAIEmbeddings(openai_api_key=configy.OPENAI_AI_KEY)

db_books = Chroma.from_documents(
    documents,
    embedding=embeddings
)

# Snowflake connection
def get_snowflake_connection():
    return snowflake.connector.connect(
        user=config.SNOWFLAKE_USER,
        password=config.SNOWFLAKE_PASSWORD,
        account=config.SNOWFLAKE_ACCOUNT,
        warehouse=config.SNOWFLAKE_WAREHOUSE,
        database=config.SNOWFLAKE_DATABASE,
        schema=config.SNOWFLAKE_SCHEMA
    )

# Account creation
def create_account(username, password):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    try:
        cs.execute(
            "INSERT INTO USER_ACCOUNTS (username, password_hash) VALUES (%s, %s)",
            (username, hashed_pw)
        )
        conn.commit()
        return "Account created successfully."
    except snowflake.connector.errors.ProgrammingError as e:
        if "unique constraint" in str(e).lower():
            return "Username already exists."
        else:
            return f"Error: {e}"
    finally:
        cs.close()
        conn.close()

# Login
def login(username, password):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "SELECT password_hash FROM USER_ACCOUNTS WHERE username = %s",
            (username,)
        )
        row = cs.fetchone()
        if row:
            stored_hash = row[0].encode()
            if bcrypt.checkpw(password.encode(), stored_hash):
                return True, "Login successful."
            else:
                return False, "Invalid password."
        else:
            return False, "User not found."
    except Exception as e:
        return False, f"Error: {e}"
    finally:
        cs.close()
        conn.close()

# Retrieve recommendations
def retrieve_recommendations(query: str, top_k: int = 5) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=top_k)
    books_list = []
    for rec in recs:
        try:
            isbn_str = rec.page_content.split()[0].strip("'").strip('"')
            isbn_num = int(isbn_str)
            books_list.append(isbn_num)
        except Exception as e:
            print(f"Skipping invalid record: {e}")
    if not books_list:
        return pd.DataFrame()
    return books[books["isbn13"].isin(books_list)].head(top_k)

# Recommend books and build dropdown
def recommend_books(description, category):
    query_parts = [description]
    if category != "All":
        query_parts.append(f"Category: {category}")
    query = ". ".join(query_parts)

    df = retrieve_recommendations(query, top_k=5)
    if df.empty:
        return "No recommendations found.", gr.update(choices=[])

    result_text = ""
    dropdown_choices = []
    for _, row in df.iterrows():
        label = f"{row['title']} by {row['authors']} (ISBN: {row['isbn13']})"
        dropdown_choices.append(label)
        result_text += (
            f"**{row['title']}** by {row['authors']}\n\n"
            f"**Genre:** {row['categories']}\n"
            f"**Average Rating:** {row['average_rating']}\n\n"
            f"{row['description'][:200]}...\n\n---\n\n"
        )
    return result_text, gr.update(choices=dropdown_choices, value=None)

# Add to reading list
def add_to_reading_list(username, isbn13):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "INSERT INTO BOOKAPPDB.PUBLIC.USER_READING_LIST (username, isbn13) VALUES (%s, %s)",
            (username, isbn13)
        )
        conn.commit()
        return f"Book {isbn13} added to your reading list."
    except Exception as e:
        return f"Error adding book: {e}"
    finally:
        cs.close()
        conn.close()

# Retrieve reading list
def get_reading_list(username):
    conn = get_snowflake_connection()
    cs = conn.cursor()
    try:
        cs.execute(
            "SELECT isbn13 FROM BOOKAPPDB.PUBLIC.USER_READING_LIST WHERE username = %s ORDER BY added_at DESC",
            (username,)
        )
        rows = cs.fetchall()
        if not rows:
            return pd.DataFrame()
        isbn_list = [r[0] for r in rows]
        return books[books["isbn13"].isin(isbn_list)]
    except Exception as e:
        print(f"Error retrieving reading list: {e}")
        return pd.DataFrame()
    finally:
        cs.close()
        conn.close()

# Format reading list as Markdown
def format_reading_list(username):
    df = get_reading_list(username)
    if df.empty:
        return "Your reading list is empty."
    text = "### Your Reading List:\n\n"
    for _, row in df.iterrows():
        text += (
            f"- **{row['title']}** by {row['authors']} (ISBN: {row['isbn13']})\n"
        )
    return text

# Login function
def login_fn(username, password):
    success, msg = login(username, password)
    if success:
        return (
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=True),
            msg,
            username
        )
    else:
        return (
            gr.update(visible=True),
            gr.update(visible=True),
            gr.update(visible=False),
            msg,
            ""
        )

# Signup function
def signup_fn(new_username, new_password):
    try:
        msg = create_account(new_username, new_password)
        return msg
    except Exception as e:
        return f"Error: {e}"

# Gradio interface
with gr.Blocks() as demo:
    user_state = gr.State()

    # Login Section
    login_section = gr.Group(visible=True)
    with login_section:
        gr.Markdown("### Login to continue")
        username = gr.Textbox(label="Username")
        password = gr.Textbox(label="Password", type="password")
        login_btn = gr.Button("Login")
        login_error = gr.Markdown("")

    # Signup Section
    signup_section = gr.Group(visible=True)
    with signup_section:
        gr.Markdown("### Create an Account")
        new_username = gr.Textbox(label="New Username")
        new_password = gr.Textbox(label="New Password", type="password")
        signup_btn = gr.Button("Create Account")
        signup_msg = gr.Markdown("")

    # App Section
    app_section = gr.Group(visible=False)
    with app_section:
        gr.Markdown("### Book Recommender")
        with gr.Row(equal_height=True):
            description = gr.Textbox(label="Describe the type of book you want")
            category = gr.Dropdown(
                label="Select a category:",
                choices=[
                    "All",
                    "History",
                    "Romance",
                    "Mystery/Thriller",
                    "Science Fiction/Fantasy",
                    "Biography/Memoir",
                    "Self-Help",
                    "Religion",
                    "Science/Technology",
                    "Philosophy",
                    "Poetry",
                    "Art",
                    "Children's",
                    "Other"
                ],
                value="All"
            )
            find_button = gr.Button("Find Recommendations")

        output = gr.Markdown()

        saved_dropdown = gr.Dropdown(
            label="Select a book to save to your reading list",
            choices=[],
            interactive=True
        )
        save_button = gr.Button("Save to Reading List")
        save_status = gr.Markdown()
        view_list_button = gr.Button("View My Reading List")
        reading_list_output = gr.Markdown()

    # Button callbacks
    login_btn.click(
        fn=login_fn,
        inputs=[username, password],
        outputs=[login_section, signup_section, app_section, login_error, user_state]
    )

    signup_btn.click(
        fn=signup_fn,
        inputs=[new_username, new_password],
        outputs=signup_msg
    )

    find_button.click(
        fn=recommend_books,
        inputs=[description, category],
        outputs=[output, saved_dropdown]
    )

    save_button.click(
        fn=lambda selected_label, username: (
            add_to_reading_list(
                username,
                int(selected_label.split("(ISBN:")[1].strip(") "))
            )
        ),
        inputs=[saved_dropdown, user_state],
        outputs=save_status
    )

    view_list_button.click(
        fn=lambda username: format_reading_list(username),
        inputs=user_state,
        outputs=reading_list_output
    )

Created a chunk of size 1170, which is longer than the specified 500
Created a chunk of size 1216, which is longer than the specified 500
Created a chunk of size 962, which is longer than the specified 500
Created a chunk of size 845, which is longer than the specified 500
Created a chunk of size 881, which is longer than the specified 500
Created a chunk of size 1191, which is longer than the specified 500
Created a chunk of size 515, which is longer than the specified 500
Created a chunk of size 754, which is longer than the specified 500
Created a chunk of size 730, which is longer than the specified 500
Created a chunk of size 723, which is longer than the specified 500
Created a chunk of size 1267, which is longer than the specified 500
Created a chunk of size 683, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 523, which is longer than the specified 500
Created a chunk of size 789, which is longer

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [38]:
# Launch app
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [36]:
demo.close()

Closing server running on port: 7860
